In [ ]:
%pip install requests
%pip install sseclient-py
%pip install pprint

In [6]:
BASE_URL = "http://localhost:8000"
API_URL = f"{BASE_URL}/api"
ACCESS_TOKEN="token"
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

In [ ]:
# create workspace
import requests
from starlette import status

workspace_id = ""
cfg = {}

payload = {
    "name": "acme3",
    "display_name": "ACME, inc.",
    "config": cfg
}

resp = requests.post(f"{API_URL}/workspaces", headers=headers, json=payload)

if resp.status_code != status.HTTP_201_CREATED:
    raise SystemExit(resp.text)

j = resp.json()
print("workspace created: ", j)
workspace_id = j['id']

In [ ]:
# get workspace
import requests
from starlette import status

resp = requests.get(f"{API_URL}/workspaces/1234", headers=headers)
if resp.status_code != status.HTTP_404_NOT_FOUND:
    raise SystemExit(f"expected code 404 but got {resp.status_code}")
resp = requests.get(f"{API_URL}/workspaces/{workspace_id}", headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
print("workspace retrieved: ", resp.json())

In [ ]:
# patch workspace
import requests
from starlette import status

payload = {
    "patch": [
        {
            "op": "replace",
            "path": "/display_name",
            "value": "Dev - ACME, inc."
        },
        {
            "op": "replace",
            "path": "/logo_url",
            "value": "https://crossid.io/logo.svg"
        }
    ]
}
resp = requests.patch(f"{API_URL}/workspaces/{workspace_id}", headers=headers, json=payload)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)

j = resp.json()
print(j)
workspace_id = j['id']


In [ ]:
# add content to workspace's vectorstore
import requests

try:
  content = """**user:erezsh**: Hi, I'm trying to run jobs in bigquery and I'm getting 403 error
          **IT**: What's the api you're trying to use?
          **user:erezsh**: `POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/my_project_id/jobs`
          **IT**: Looks like you need an iam role. I'll give you the role **roles/bigquery.admin**. You can try again now.
          **user:erezsh**: That worked, thanks."""
  
  data = {
    "docs": 
      [
        {
           "content": content,
           "directory": "dir1",
           "apps": ["a1", "a2"],
           "external_id": "1234",
        }
      ]
  }

  response = requests.post(f"{API_URL}/content", headers=headers, json=data)
  if response.status_code >= 200 and response.status_code <= 399:
      print("content successfully added: ", response.text)
  else:
      print(f"failed to add content with code: {response.status_code} || {response.text}")

except requests.RequestException as e:
  print(f"An error occurred: {e}")

In [ ]:
# update content
import requests

try:
  content = "replaced to: foo bar"
  
  data = {
    "docs": 
      [
        {
           "content": content,
           "directory": "dir1",
           "apps": ["a1"],
           "external_id": "1234",
        }
      ]
  }

  response = requests.put(f"{API_URL}/content", headers=headers, json=data)
  if response.status_code >= 200 and response.status_code <= 399:
      print("content successfully replaced: ", response.text)
  else:
      print(f"failed to add content with code: {response.status_code} || {response.text}")

except requests.RequestException as e:
  print(f"An error occurred: {e}")

In [ ]:
# get doc
import json
import requests

try:
  url = f"{API_URL}/content/1234"
  qp = {
     "projection": ["apps", "directory"]
  }
  response = requests.get(url=url, params=qp, headers=headers)
  if response.status_code >= 200 and response.status_code <= 399:
      print(json.dumps(response.json(), indent=4))
  else:
      print(f"failed to get doc with code: {response.status_code} || {response.text}")

except requests.RequestException as e:
  print(f"An error occurred: {e}")

In [ ]:
# list docs
import json
import requests

try:
  url = f"{API_URL}/content"
  qp = {
     "app_names": ["a1"],
     "projection": ["apps", "directory"]
  }
  response = requests.get(url=url, params=qp, headers=headers)
  if response.status_code >= 200 and response.status_code <= 399:
      print(json.dumps(response.json(), indent=4))
  else:
      print(f"failed to list docs with code: {response.status_code} || {response.text}")

except requests.RequestException as e:
  print(f"An error occurred: {e}")

In [ ]:
# remove data
import requests

try:
  data = {
     "ids": ["1234"]
  }

  response = requests.post(f"{API_URL}/content/.delete", headers=headers, json=data)
  if response.status_code >= 200 and response.status_code <= 399:
      print("content successfully removed: ", response.text)
  else:
      print(f"failed to add content with code: {response.status_code} || {response.text}")

except requests.RequestException as e:
  print(f"An error occurred: {e}")

In [ ]:
# create conversation
import requests
from starlette import status


payload = {
    # "external_id": "foo"
}

resp = requests.post(f"{API_URL}/conversations", headers=headers, json=payload)

if resp.status_code != status.HTTP_201_CREATED:
    raise SystemExit(f"{resp.status_code}: {resp.json()['detail']}")

j = resp.json()
print("conversation created: ", j)
conversation_id = j['id']

In [ ]:
# send a message to conversation
import json
import requests
from sseclient import SSEClient

def converse(input: str):
    data = {
        "input": input
    }
    _headers = headers.copy()
    _headers["Accept"] = "text/event-stream"
    with requests.post(
        url=f"{API_URL}/conversations/{conversation_id}/stream",
        json=data,
        headers=_headers,
        stream=True
    ) as r:
        if r.status_code != 200:
            raise SystemExit(f"expected status 200 but got {r.status_code}: {r.text}")

        client = SSEClient(r)
        current_event_type = ''
        current_name = ''
        for event in client.events():
            event_type = event.event
            data = json.loads(event.data)
            c = data['content']
            name = data['name']
            if name != current_name:
                print("\n--")
                current_name = name
                current_event_type = ''
                print(f"{name}:")

            if current_event_type != event_type:
                current_event_type = event_type
                print(f"\n(event_type){event_type}:")

            print(c, end='', flush=True)
        

try:
    input = "Hi, I need access to Jira"
    converse(input)

    # input = "I need to run jobs"
    # converse(input)
    
except requests.RequestException as e:
    print(f"error: {e}")

In [ ]:
# get conversation
import requests
from starlette import status
from pprint import pprint
    
resp = requests.get(f"{API_URL}/requests/404", headers=headers)
if resp.status_code != status.HTTP_404_NOT_FOUND:
    raise SystemExit(f"expected code 404 but got {resp.status_code}")
    
resp = requests.get(f"{API_URL}/conversations/{conversation_id}?links=messages", headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
pprint(resp.json())


In [ ]:
# list conversations for user
import requests
from starlette import status
from pprint import pprint

qp = {
    # 'links': 'messages',
    'limit': 1,
    'q': 'status eq "active"',
    "projection": ["id"]
}
resp = requests.get(f"{API_URL}/conversations", params=qp, headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
pprint(resp.json())


In [ ]:
# archive conversation
import requests
from starlette import status
from pprint import pprint

resp = requests.post(f"{API_URL}/conversations/{conversation_id}/.archive", headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
pprint(resp.json())


In [ ]:
# unarchive conversation
import requests
from starlette import status
from pprint import pprint
    
resp = requests.post(f"{API_URL}/conversations/{conversation_id}/.unarchive", headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
pprint(resp.json())


In [ ]:
# cancel conversation
import requests
from starlette import status
from pprint import pprint
    
resp = requests.post(f"{API_URL}/conversations/{conversation_id}/.cancel", headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
pprint(resp.json())


In [ ]:
# delete workspace
import requests
from starlette import status
workspace_id="IlQ57G6N8B"
resp = requests.delete(f"{API_URL}/workspaces/{workspace_id}", headers=headers)
if resp.status_code != status.HTTP_204_NO_CONTENT:
    raise SystemExit(resp.text)
print("workspace deleted successfully")

In [ ]:
# create application
import requests
from starlette import status

payload = {
    "name": "projects",
    "aliases": ["foo", "jira"],
    "extra_instructions": "some more instructions",
    "provision_schema": {
        "project_name": {"description": "the project name"}
    }
}

resp = requests.post(f"{API_URL}/applications", headers=headers, json=payload)

if resp.status_code != status.HTTP_201_CREATED:
    raise SystemExit(resp.text)

j = resp.json()
print("application created: ", j)
app_id = j['id']

payload2 = {
    "name": "projects",
    "aliases": [],
    "extra_instructions": "some more instructions"
}
# make sure unique constrain works
resp = requests.post(f"{API_URL}/applications", headers=headers, json=payload2)

if resp.status_code != status.HTTP_409_CONFLICT:
    # unique constrain doesn't work
    raise SystemExit(resp.text)

In [ ]:
# update application

import requests
from starlette import status

payload = {
    "patch": [
        {
            "op": "replace",
            "path": "/aliases",
            "value": ["bar", "jira"]
        }
    ]
}
resp = requests.patch(f"{API_URL}/applications/{app_id}", headers=headers, json=payload)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)

j = resp.json()
print(j)
app_id = j['id']


In [ ]:
# list applications

import requests
from starlette import status
from pprint import pprint

url = f"{API_URL}/applications"
qp = {
    "limit": 1,
    "offset": 0,
    "projection": ["name", "id"]
}
resp = requests.get(url=url, params=qp, headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
pprint(resp.json())

In [ ]:
# delete application
resp = requests.delete(f"{API_URL}/applications/{app_id}", headers=headers, json=payload)
if resp.status_code != status.HTTP_204_NO_CONTENT:
    raise SystemExit(resp.text)

In [ ]:
# create a directory
import requests
from starlette import status

payload = {
    "name": "projects",
    "provisioning_config": {
        "type": "webhook",
        "config": {
            "url": "https://example.com",
        }
    }
}

resp = requests.post(f"{API_URL}/directories", headers=headers, json=payload)

if resp.status_code != status.HTTP_201_CREATED:
    raise SystemExit(resp.text)

j = resp.json()
print("directory created: ", j)
dir_id = j['id']

payload2 = {
    "name": "projects",
    "aliases": [],
    "extra_instructions": "some more instructions"
}
# make sure unique constrain works
resp = requests.post(f"{API_URL}/directories", headers=headers, json=payload2)

if resp.status_code != status.HTTP_409_CONFLICT:
    # unique constrain doesn't work
    raise SystemExit(resp.text)

In [ ]:
# update directory

import requests
from starlette import status

payload = {
    "patch": [
        {
            "op": "replace",
            "path": "/provisioning_config",
            "value": {
                "type": "webhook",
                "config": {
                    "url": "http://localhost:9005/api/grant"
                }
            }
        }
    ]
}
resp = requests.patch(f"{API_URL}/directories/{dir_id}", headers=headers, json=payload)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)

j = resp.json()
print(j)


In [ ]:
# get a directory
import requests
from starlette import status

resp = requests.get(f"{API_URL}/directories/1234", headers=headers)
if resp.status_code != status.HTTP_404_NOT_FOUND:
    raise SystemExit(f"expected code 404 but got {resp.status_code}")
resp = requests.get(f"{API_URL}/directories/{dir_id}", headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
print("directory retrieved: ", resp.json())

In [ ]:
# list directories

import requests
from starlette import status
from pprint import pprint

url = f"{API_URL}/directories"
qp = {
    "limit": 1,
    "offset": 0,
    "projection": ["name", "id"]
}
resp = requests.get(url=url, params=qp, headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)
pprint(resp.json())

In [ ]:
# delete directory
resp = requests.delete(f"{API_URL}/directories/{dir_id}", headers=headers, json=payload)
if resp.status_code != status.HTTP_204_NO_CONTENT:
    raise SystemExit(resp.text)

In [ ]:
# delete workspace
import requests
from starlette import status
workspace_id="IlQ57G6N8B"
resp = requests.delete(f"{API_URL}/workspaces/{workspace_id}", headers=headers)
if resp.status_code != status.HTTP_204_NO_CONTENT:
    raise SystemExit(resp.text)
print("workspace deleted successfully")

In [ ]:
# import okta roles
dir_id = "ABCDE11ere"
resp = requests.post(f"{API_URL}/directories/{dir_id}/.import", headers=headers, json={"create_apps": True})
if resp.status_code != status.HTTP_202_ACCEPTED:
    raise SystemExit(resp.text)
print("content being processed: ", resp.json())

In [ ]:
# create rule
import requests
from starlette import status

payload = {
    "when": "foo is bar",
    "then": "approve",
    "type": "auto_approve",
    "application_ids": ["yhki9dP5u9"]
}

resp = requests.post(f"{API_URL}/rules", headers=headers, json=payload)

if resp.status_code != status.HTTP_201_CREATED:
    raise SystemExit(resp.text)

j = resp.json()
print("rule created: ", j)
if not j["active"]:
    raise SystemExit("rule should be active")

rule_id = j['id']

In [ ]:
# get rule
import requests
from starlette import status

resp = requests.get(f"{API_URL}/rules/{rule_id}", headers=headers, json=payload)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)

j = resp.json()
print(j)
app_id = j['id']

In [ ]:
# update rule
import requests
from starlette import status

payload = {
    "patch": [
        {
            "op": "replace",
            "path": "/when",
            "value": "foo lower then 2"
        },
        {
            "op": "replace",
            "path": "/active",
            "value": False
        }
    ]
}
resp = requests.patch(f"{API_URL}/rules/{rule_id}", headers=headers, json=payload)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)

j = resp.json()
print(j)
if j["active"]:
    raise SystemExit("rule should be deactivated")

rule_id = j['id']

In [ ]:
# list rules
import requests
from starlette import status

resp = requests.get(f"{API_URL}/rules", headers=headers, json=payload)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)

j = resp.json()
print(j)

In [ ]:
# delete rule
import requests
from starlette import status

resp = requests.delete(f"{API_URL}/rules/{rule_id}", headers=headers)
if resp.status_code != status.HTTP_204_NO_CONTENT:
    raise SystemExit(resp.text)
print("rule deleted successfully")

In [ ]:
# install gmail app
import requests
from starlette import status

payload = {
    "project_id": "project-id",
    "topic_id": "topic-id",
    "email_address": "some@address.com",
    "client_id": "1234",
    "client_secret": "1234",
}

resp = requests.post(f"{BASE_URL}/email_communication/google_cloud/install", headers=headers, json=payload)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)

print(resp.json())

In [ ]:
# start watch
import requests
from starlette import status

resp = requests.get(f"{BASE_URL}/email_communication/watch", headers=headers)
if resp.status_code != status.HTTP_200_OK:
    raise SystemExit(resp.text)

print(resp.json())